# Paso 2: Selección y Almacenamiento de Datos Relevantes

## Introducción

Luego de realizar la exploración inicial de la API de Mercado Libre, se procede a aplicar una metodología de ciencia de datos para identificar y almacenar los datos que se utilizarán en el análisis. En este challenge se ha decidido trabajar con la subcategoría **"Otros"** (ID: MCO1070), ya que agrupa productos sin una clasificación específica, lo que representa un desafío interesante para identificar similitudes entre ítems heterogéneos. Además, considerando la limitación de la API a un máximo de 1000 productos por consulta, se optará por extraer los 1000 ítems más vendidos (según `sold_quantity`) que tengan stock disponible. Esto permitirá trabajar con una muestra manejable y representativa del universo de productos de esta subcategoría.

## Metodología para la Selección de Datos

1. **Comprensión del Dominio y del Problema:**
   - **Objetivo:**  
     Identificar y agrupar productos similares o idénticos, incluso cuando son vendidos por distintos sellers, para mejorar la experiencia del usuario al facilitar la comparación.
   - **Contexto de la Subcategoría "Otros":**  
     Los productos en "Otros" no cuentan con una clasificación detallada, lo que dificulta la agrupación automática. Por ello, es esencial extraer información que permita identificar similitudes basadas en los datos disponibles.

2. **Selección y Filtrado de Campos:**
   - **Análisis Exploratorio:**  
     Se revisa el DataFrame inicial (por ejemplo, usando `df.head()`) para identificar los campos presentes. Se determina que los siguientes campos son fundamentales para el análisis:
   - **Campos de Texto:**
     - `id`: Identificador único del producto, útil para la referencia y validación.
     - `title`: Título del producto. Debido a que el campo `sanitized_title` generalmente está vacío, se utiliza el `title` original para el análisis textual.
     - *Opcionalmente*, si se cuenta con `description` o `subtitle`, estos pueden aportar contexto adicional.
   - **Campos de Imagen:**
     - `thumbnail`: URL de la imagen principal, esencial para extraer características visuales mediante modelos pre-entrenados (por ejemplo, ResNet o VGG).
   - **Otros Campos Complementarios:**
     - `price`: Permite segmentar y comparar productos por su rango de precio.
     - `sold_quantity`: Indicador de popularidad, utilizado para ordenar los productos y seleccionar los más vendidos.
     - `available_quantity`: Fundamental para filtrar productos con stock disponible.
     - `permalink`: Enlace directo al producto, facilitando la validación manual.

3. **Almacenamiento de los Datos Seleccionados:**
   - **Formato Estructurado:**  
     Se recomienda almacenar los datos en un DataFrame de Pandas, lo que facilita el manejo, la limpieza y el preprocesamiento de la información.
   - **Persistencia:**  
     Aunque inicialmente se mantendrán los datos en memoria para el análisis exploratorio, en fases posteriores se podrán guardar en formatos como CSV o en bases de datos para permitir la replicación y análisis incremental.

4. **Preprocesamiento Inicial:**
   - **Para los Datos Textuales:**  
     Se limpiarán y normalizarán los títulos (conversión a minúsculas, eliminación de puntuación y stopwords), y se convertirán en vectores mediante técnicas como TF-IDF o embeddings para medir similitudes semánticas entre productos.
   - **Para las Imágenes:**  
     Se validarán las URLs y, en etapas posteriores, se utilizarán modelos pre-entrenados (como ResNet o VGG) para extraer embeddings que permitan comparar productos de forma visual.

## Conclusión

La estrategia de selección y almacenamiento se centra en extraer y trabajar únicamente con la información que aporta valor al análisis. Al enfocarse en la subcategoría **"Otros"** —que agrupa productos sin una clasificación específica— y extrayendo los 1000 productos más vendidos con stock disponible, se garantiza una muestra representativa y manejable para aplicar técnicas de preprocesamiento, medición de similitud y clustering. Este enfoque permite afrontar el desafío de agrupar ítems similares en un contexto real, optimizando la experiencia del usuario al ofrecer opciones comparables en el marketplace.


In [2]:
import requests

# Definir la categoría a consultar (por ejemplo, "Electrónica, Audio y Video" en Colombia)
CATEGORY_ID = "MCO1000"
API_URL = f"https://api.mercadolibre.com/sites/MCO/search?category={CATEGORY_ID}&offset=0"

# Realizar la consulta a la API
response = requests.get(API_URL)

if response.status_code == 200:
    data = response.json()
    # El campo 'total' se encuentra dentro de 'paging'
    total_productos = data.get("paging", {}).get("total", 0)
    print("Cantidad total de productos en la categoría:", total_productos)
else:
    print("Error en la consulta:", response.status_code)


Cantidad total de productos en la categoría: 920968


In [3]:
import requests

# ID de la categoría principal "Electrónica, Audio y Video" en Colombia
CATEGORY_ID = "MCO1000"

# Obtener subcategorías dentro de "Electrónica, Audio y Video"
url = f"https://api.mercadolibre.com/categories/{CATEGORY_ID}"
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    category_data = response.json()
    subcategories = category_data.get("children_categories", [])

    # Mostrar las subcategorías disponibles
    for subcat in subcategories:
        print(f"ID: {subcat['id']} | Nombre: {subcat['name']} | Total ítems: {subcat['total_items_in_this_category']}")

else:
    print(f"Error al obtener subcategorías: {response.status_code}")


ID: MCO3690 | Nombre: Accesorios Audio y Video | Total ítems: 69026
ID: MCO431414 | Nombre: Accesorios para TV | Total ítems: 38198
ID: MCO3835 | Nombre: Audio | Total ítems: 426937
ID: MCO5054 | Nombre: Cables | Total ítems: 156003
ID: MCO11830 | Nombre: Componentes Electrónicos | Total ítems: 219639
ID: MCO4632 | Nombre: Controles Remotos | Total ítems: 23099
ID: MCO173235 | Nombre: Drones y Accesorios | Total ítems: 42144
ID: MCO442042 | Nombre: Fundas y Bolsos | Total ítems: 4897
ID: MCO176837 | Nombre: Media Streaming | Total ítems: 5794
ID: MCO4102 | Nombre: Pilas y Cargadores | Total ítems: 51488
ID: MCO419930 | Nombre: Repuestos TV | Total ítems: 20923
ID: MCO14903 | Nombre: Televisores | Total ítems: 2684
ID: MCO442056 | Nombre: Video | Total ítems: 12615
ID: MCO4800 | Nombre: Video Beams y Pantallas | Total ítems: 37727
ID: MCO1070 | Nombre: Otros | Total ítems: 7577


In [4]:
import requests

# ID de la categoría "Otros" dentro de Electrónica, Audio y Video
CATEGORY_ID = "MCO1070"

# URL para obtener las subcategorías de la categoría "Otros"
url = f"https://api.mercadolibre.com/categories/{CATEGORY_ID}"
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    category_data = response.json()
    subcategories = category_data.get("children_categories", [])
    
    # Verificar si hay subcategorías y mostrarlas
    if subcategories:
        print(f"Subcategorías dentro de la categoría '{category_data['name']}' (ID: {CATEGORY_ID}):\n")
        for subcat in subcategories:
            print(f"ID: {subcat['id']} | Nombre: {subcat['name']} | Total ítems: {subcat['total_items_in_this_category']}")
    else:
        print(f"La categoría '{category_data['name']}' (ID: {CATEGORY_ID}) no tiene subcategorías.")
else:
    print(f"Error al consultar la categoría '{CATEGORY_ID}': {response.status_code}")


La categoría 'Otros' (ID: MCO1070) no tiene subcategorías.


# Justificación de la Elección de la Subcategoría "Otros" (ID: MCO1070)

## Contexto y Problema

En el proceso de extracción y análisis de productos del marketplace, se observó que existen numerosas publicaciones que no encajan en una clasificación definida, lo que las agrupa dentro de la subcategoría **"Otros"**. Esta agrupación representa productos cuya clasificación específica no ha sido determinada o que, por su naturaleza heterogénea, no se ajustan a las categorías tradicionales.

1. **Falta de Clasificación Específica:**
   - Los productos incluidos en "Otros" carecen de una clasificación clara, lo que puede dificultar la comparación directa y el análisis de similitud cuando se combinan con ítems de categorías más definidas.
   - Al concentrarse en esta subcategoría, se puede focalizar el análisis en un conjunto heterogéneo de productos, lo que permite identificar patrones y aplicar técnicas de agrupamiento para encontrar similitudes entre ítems que, a primera vista, pueden parecer dispares.

2. **Limitación de la API (Máximo 1000 Productos por Consulta):**
   - La API de Mercado Libre limita la extracción a un máximo de 1000 productos por consulta. Esta restricción es especialmente relevante en categorías con un alto volumen de publicaciones.
   - Al elegir la subcategoría "Otros", se aprovecha la estrategia de obtener los 1000 productos más vendidos (o más representativos) que tienen stock disponible. Esto permite trabajar con una muestra manejable y representativa de un universo potencialmente muy amplio.
   - La extracción se realizará en bloques (paginación) hasta alcanzar los 1000 ítems, garantizando así que se cuenta con una muestra consistente y comparable, que a su vez se utilizará en el análisis de agrupamiento y en la comparación de productos similares.

## Enfoque Propuesto

- **Extracción y Filtrado:**
  - Se extraerán 1000 ítems de la subcategoría "Otros" (*ID: MCO1070*), aplicando el criterio de seleccionar solo aquellos productos que tienen stock disponible.
  - Se ordenarán los resultados por la cantidad de ventas (`sold_quantity_desc`), de forma que se obtengan los 1000 productos más vendidos dentro de esta subcategoría.

- **Análisis y Agrupamiento:**
  - Una vez obtenida la muestra de 1000 productos, se procederá a realizar el preprocesamiento (técnicas de NLP para los títulos, extracción de características de imágenes, etc.) y se aplicarán algoritmos de clustering para identificar ítems similares.
  - Este enfoque permite, pese a la heterogeneidad inherente a la subcategoría "Otros", agrupar y comparar productos que pueden estar ofreciendo funciones o características similares, mejorando así la experiencia del usuario al presentar opciones comparables.


In [18]:
import requests
import pandas as pd
import time
from tqdm import tqdm

# Definir la subcategoría a analizar: 'Otros' de la categoría Electrónica, Audio y Video en Colombia
CATEGORY_ID = "MCO1070"  # ID de la subcategoría elegida

# Número objetivo de productos a extraer
TARGET_COUNT = 1000
OFFSET_STEP = 50  # La API devuelve datos en bloques de 50

# Lista para almacenar los productos que cumplen la condición (con stock disponible)
products = []
offset = 0

# URL base de la API con ordenamiento por sold_quantity_desc (más vendidos primero)
API_URL_BASE = f"https://api.mercadolibre.com/sites/MCO/search?category={CATEGORY_ID}&sort=sold_quantity_desc&offset="

# Configurar tqdm para mostrar el progreso basado en la cantidad de productos deseados
with tqdm(total=TARGET_COUNT, desc="Extrayendo productos") as pbar:
    while len(products) < TARGET_COUNT:
        url = API_URL_BASE + str(offset)
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Error en la consulta de offset {offset}: {response.status_code}")
            break
        
        data = response.json()
        results = data.get("results", [])
        if not results:
            # Si no se reciben resultados, se sale del bucle
            break
        
        # Filtrar ítems que tengan stock disponible (available_quantity > 0)
        available_items = [item for item in results if item.get("available_quantity", 0) > 0]
        
        # Guardar el número previo de productos para calcular la cantidad nueva
        prev_count = len(products)
        products.extend(available_items)
        new_count = len(products) - prev_count
        
        # Actualizar la barra de progreso con la cantidad de ítems nuevos (hasta TARGET_COUNT)
        pbar.update(new_count)
        
        # Incrementar el offset para la siguiente página
        offset += OFFSET_STEP
        time.sleep(0.5)  # Pausa para evitar saturar la API

# Limitar la lista a los TARGET_COUNT productos
top_items = products[:TARGET_COUNT]

# Convertir la lista de productos a un DataFrame para análisis posterior
df = pd.json_normalize(top_items)

print(f"\nTotal de productos que cumplen la condición: {len(top_items)}")
print("Muestra de los primeros registros del DataFrame:")
print(df.head())


Extrayendo productos: 100%|██████████| 1000/1000 [00:15<00:00, 66.37it/s]


Total de productos que cumplen la condición: 1000
Muestra de los primeros registros del DataFrame:
              id                                              title condition  \
0  MCO1312345739                     Cerca 75km - Ganado - Impulsor       new   
1  MCO1401210195      Planta Cerca Electrica Para Ganado 50km M. Jr       new   
2   MCO601595467  Limpiador  Electrónico -removedor De Polvo - A...       new   
3   MCO611865640  Planta Cerca Eléctrica Solar 100km, Incluye Pa...       new   
4   MCO599963912  Tensor Metálico Para Cerca Eléctrica Para 500 ...       new   

                   thumbnail_id catalog_product_id listing_type_id  \
0  926804-MCO70350738436_072023               None        gold_pro   
1  802783-MCO74899505438_032024               None        gold_pro   
2  995727-MCO73203873773_122023               None    gold_special   
3  844953-MCO45203598828_032021               None        gold_pro   
4  797256-MCO71264690908_082023               None    gold_spec

In [19]:
df

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,sanitized_title,permalink,buying_mode,site_id,...,variations_data.54303203436.attributes,variations_data.54303203436.attribute_combinations,variations_data.54303203432.thumbnail,variations_data.54303203432.ratio,variations_data.54303203432.name,variations_data.54303203432.pictures_qty,variations_data.54303203432.price,variations_data.54303203432.user_product_id,variations_data.54303203432.attributes,variations_data.54303203432.attribute_combinations
0,MCO1312345739,Cerca 75km - Ganado - Impulsor,new,926804-MCO70350738436_072023,None,gold_pro,,https://articulo.mercadolibre.com.co/MCO-13123...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MCO1401210195,Planta Cerca Electrica Para Ganado 50km M. Jr,new,802783-MCO74899505438_032024,None,gold_pro,,https://articulo.mercadolibre.com.co/MCO-14012...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MCO601595467,Limpiador Electrónico -removedor De Polvo - A...,new,995727-MCO73203873773_122023,None,gold_special,,https://articulo.mercadolibre.com.co/MCO-60159...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MCO611865640,"Planta Cerca Eléctrica Solar 100km, Incluye Pa...",new,844953-MCO45203598828_032021,None,gold_pro,,https://articulo.mercadolibre.com.co/MCO-61186...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MCO599963912,Tensor Metálico Para Cerca Eléctrica Para 500 ...,new,797256-MCO71264690908_082023,None,gold_special,,https://articulo.mercadolibre.com.co/MCO-59996...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,MCO542097832,Gripple Plus Medium Pack De 20 | Alambre Joine...,new,926412-MCO44224848873_122020,None,gold_special,,https://articulo.mercadolibre.com.co/MCO-54209...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,MCO542414089,Presurizador Bomba Automatico Soluciona La Baj...,new,914137-MCO45760245009_042021,None,gold_pro,,https://articulo.mercadolibre.com.co/MCO-54241...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,MCO542469534,"8,5 X 11 Sostenedor De La Muestra De Acrílico ...",new,864629-MCO40475967701_012020,None,gold_special,,https://articulo.mercadolibre.com.co/MCO-54246...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,MCO542656528,Fantasia Walt Disney Video Laserdisc- Musical ...,used,841362-MCO28010690664_082018,None,gold_special,,https://articulo.mercadolibre.com.co/MCO-54265...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Estadistica descriptiva de los datos elegidos

In [20]:
import pandas as pd


# 1. Visualizar todas las columnas disponibles en el DataFrame
print("Columnas disponibles en el DataFrame:")
print(df.columns.tolist())

# 2. Información general del DataFrame
print("\nInformación general del DataFrame:")
print(df.info())

# 3. Distribución de la condición del producto
print("\nDistribución del campo 'condition':")
print(df['condition'].value_counts())

# 4. Ejemplo de exploración del campo 'attributes'
# Este campo puede contener información compleja (por ejemplo, listas o diccionarios)
print("\nEjemplo del contenido del campo 'attributes' del primer producto:")
print(df['attributes'].iloc[0])


Columnas disponibles en el DataFrame:
['id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id', 'listing_type_id', 'sanitized_title', 'permalink', 'buying_mode', 'site_id', 'category_id', 'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price', 'original_price', 'available_quantity', 'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago', 'stop_time', 'attributes', 'winner_item_id', 'catalog_listing', 'discounts', 'promotion_decorations', 'promotions', 'inventory_id', 'sale_price.price_id', 'sale_price.amount', 'sale_price.conditions.eligible', 'sale_price.conditions.context_restrictions', 'sale_price.conditions.start_time', 'sale_price.conditions.end_time', 'sale_price.currency_id', 'sale_price.exchange_rate', 'sale_price.payment_method_prices', 'sale_price.payment_method_type', 'sale_price.regular_amount', 'sale_price.type', 'shipping.store_pick_up', 'shipping.free_shipping', 'shipping.logistic_type', 'shipping.mode', 'shipping.tags', 'shipping.benefits', '

In [21]:
import pandas as pd

# Función recursiva para convertir objetos en estructuras hashables
def make_hashable(x):
    if isinstance(x, list):
        # Convertir cada elemento de la lista recursivamente y devolver una tupla
        return tuple(make_hashable(e) for e in x)
    elif isinstance(x, dict):
        # Convertir el diccionario en una tupla de pares (clave, valor), ordenados por clave
        return tuple(sorted((k, make_hashable(v)) for k, v in x.items()))
    else:
        return x

# Aplicar la conversión a cada columna para luego calcular nunique
unique_counts = df.apply(lambda col: col.apply(make_hashable).nunique())

# Crear un DataFrame resumen con información de cada columna
summary = pd.DataFrame({
    'dtype': df.dtypes,
    'missing_count': df.isnull().sum(),
    'missing_ratio': df.isnull().mean(),   # Proporción de valores nulos
    'unique_count': unique_counts           # Número de valores únicos, tratando listas y dicts como hashables
})

# Ordenar el resumen de mayor a menor proporción de valores nulos
summary_sorted = summary.sort_values(by='missing_ratio', ascending=False)
print("Resumen de columnas (ordenado por porcentaje de valores nulos):")
print(summary_sorted)

# Definir umbrales para filtrar columnas
missing_threshold = 0.8  # Por ejemplo, descartar columnas con más del 80% de valores nulos
unique_threshold = 1     # Columnas constantes (único valor)

# Identificar columnas con alto porcentaje de valores nulos
columns_high_missing = summary[summary['missing_ratio'] > missing_threshold].index.tolist()
print(f"\nColumnas con más del {missing_threshold*100}% de valores nulos:")
print(columns_high_missing)

# Identificar columnas que son constantes (o casi, según el umbral deseado)
columns_constant = summary[summary['unique_count'] <= unique_threshold].index.tolist()
print("\nColumnas constantes (con 1 o menos valores únicos):")
print(columns_constant)

# Opcional: Combinar ambas listas y eliminar estas columnas del DataFrame
columns_to_drop = set(columns_high_missing + columns_constant)
df_filtered = df.drop(columns=columns_to_drop)

print("\nShape del DataFrame original:", df.shape)
print("Shape del DataFrame después de eliminar columnas vacías/constantes:", df_filtered.shape)


Resumen de columnas (ordenado por porcentaje de valores nulos):
                           dtype  missing_count  missing_ratio  unique_count
winner_item_id            object           1000            1.0             0
sale_price.exchange_rate  object           1000            1.0             0
promotion_decorations     object           1000            1.0             0
promotions                object           1000            1.0             0
discounts                 object           1000            1.0             0
...                          ...            ...            ...           ...
listing_type_id           object              0            0.0             2
thumbnail_id              object              0            0.0           956
condition                 object              0            0.0             2
title                     object              0            0.0           967
id                        object              0            0.0          1000

[627 rows x

In [22]:
df_filtered.head()

,id,title,condition,thumbnail_id,listing_type_id,permalink,thumbnail,order_backend,price,available_quantity,...,shipping.mode,shipping.tags,seller.id,seller.nickname,address.state_id,address.state_name,address.city_id,address.city_name,installments.quantity,installments.amount
0,MCO1312345739,Cerca 75km - Ganado - Impulsor,new,926804-MCO70350738436_072023,gold_pro,https://articulo.mercadolibre.com.co/MCO-13123...,http://http2.mlstatic.com/D_926804-MCO70350738...,1,165000,1,...,me2,"[fulfillment, mandatory_free_shipping]",362243902,CERCASDELORIENTE,CO-ANT,Antioquia,Q08tQU5URWzCoFNhbnR1YXJpbw,El Santuario,12.0,13750.0
1,MCO1401210195,Planta Cerca Electrica Para Ganado 50km M. Jr,new,802783-MCO74899505438_032024,gold_pro,https://articulo.mercadolibre.com.co/MCO-14012...,http://http2.mlstatic.com/D_802783-MCO74899505...,2,140000,1,...,me2,[mandatory_free_shipping],473010128,PROELECTRODECOLOMBIA,CO-ANT,Antioquia,TUNPQ01BUjdjNWY5,Marinilla,12.0,11667.0
2,MCO601595467,Limpiador Electrónico -removedor De Polvo - A...,new,995727-MCO73203873773_122023,gold_special,https://articulo.mercadolibre.com.co/MCO-60159...,http://http2.mlstatic.com/D_995727-MCO73203873...,3,21500,1,...,me2,[self_service_out],36449755,HRICARDO13,CO-VAC,Valle Del Cauca,TUNPQ0NBTDYyZDA0,Cali,3.0,7167.0
3,MCO611865640,"Planta Cerca Eléctrica Solar 100km, Incluye Pa...",new,844953-MCO45203598828_032021,gold_pro,https://articulo.mercadolibre.com.co/MCO-61186...,http://http2.mlstatic.com/D_844953-MCO45203598...,4,690000,1,...,me2,[mandatory_free_shipping],74243544,VICTORJAIMED,CO-BOY,Boyaca,TUNPQ0RVSTJiYmU5,Duitama,12.0,57500.0
4,MCO599963912,Tensor Metálico Para Cerca Eléctrica Para 500 ...,new,797256-MCO71264690908_082023,gold_special,https://articulo.mercadolibre.com.co/MCO-59996...,http://http2.mlstatic.com/D_797256-MCO71264690...,5,6600,250,...,me2,[],44135937,LUXURY SOLAR,CO-NAR,Nariño,TUNPQ0lQSTY0ZTE2,Ipiales,NaN,NaN


In [23]:
df_filtered.columns

Index(['id', 'title', 'condition', 'thumbnail_id', 'listing_type_id',
       'permalink', 'thumbnail', 'order_backend', 'price',
       'available_quantity', 'use_thumbnail_id', 'stop_time', 'attributes',
       'sale_price.amount', 'sale_price.conditions.context_restrictions',
       'sale_price.type', 'shipping.free_shipping', 'shipping.logistic_type',
       'shipping.mode', 'shipping.tags', 'seller.id', 'seller.nickname',
       'address.state_id', 'address.state_name', 'address.city_id',
       'address.city_name', 'installments.quantity', 'installments.amount'],
      dtype='object')

# Selección de Campos Pertinentes para el Análisis

En el contexto del challenge, el objetivo es identificar y agrupar productos similares o idénticos en el marketplace. Para ello, se recomienda centrarse en los campos que permiten realizar comparaciones tanto desde el punto de vista textual como visual, y que aporten atributos relevantes del producto.

## Campos Seleccionados y Justificación

- **id:**  
  Permite identificar de manera única cada producto y realizar seguimiento a lo largo del análisis.

- **title**  
  - **title:** Contiene el título original del producto, que es esencial para evaluar la descripción y detectar similitudes semánticas.  

- **attributes:**  
  Contiene información adicional relevante (como marca, modelo, etc.) que puede ser crucial para confirmar la similitud entre productos, especialmente cuando los títulos pueden variar ligeramente.

- **thumbnail:**  
  Proporciona la URL de la imagen principal del producto, la cual se utilizará para el análisis visual mediante la extracción de features (por ejemplo, usando modelos pre-entrenados de visión por computadora).

- **price :**  
  El precio es un dato útil para filtrar o agrupar productos, ya que productos similares suelen tener rangos de precio cercanos.  


In [24]:
pertinent_columns = ['id', 'title', 'attributes', 'thumbnail', 'price']
df_pertinent = df_filtered[pertinent_columns]
df_pertinent

,id,title,attributes,thumbnail,price
0,MCO1312345739,Cerca 75km - Ganado - Impulsor,"[{'id': 'BRAND', 'name': ' Marca del producto ...",http://http2.mlstatic.com/D_926804-MCO70350738...,165000
1,MCO1401210195,Planta Cerca Electrica Para Ganado 50km M. Jr,"[{'id': 'BRAND', 'name': ' Marca del producto ...",http://http2.mlstatic.com/D_802783-MCO74899505...,140000
2,MCO601595467,Limpiador Electrónico -removedor De Polvo - A...,"[{'id': 'BRAND', 'name': ' Marca del producto ...",http://http2.mlstatic.com/D_995727-MCO73203873...,21500
3,MCO611865640,"Planta Cerca Eléctrica Solar 100km, Incluye Pa...","[{'id': 'BRAND', 'name': ' Marca del producto ...",http://http2.mlstatic.com/D_844953-MCO45203598...,690000
4,MCO599963912,Tensor Metálico Para Cerca Eléctrica Para 500 ...,"[{'id': 'BRAND', 'name': ' Marca del producto ...",http://http2.mlstatic.com/D_797256-MCO71264690...,6600
...,...,...,...,...,...
995,MCO542097832,Gripple Plus Medium Pack De 20 | Alambre Joine...,"[{'id': 'BRAND', 'name': ' Marca del producto ...",http://http2.mlstatic.com/D_926412-MCO44224848...,274000
996,MCO542414089,Presurizador Bomba Automatico Soluciona La Baj...,"[{'id': 'BRAND', 'name': ' Marca del producto ...",http://http2.mlstatic.com/D_914137-MCO45760245...,379999
997,MCO542469534,"8,5 X 11 Sostenedor De La Muestra De Acrílico ...","[{'id': 'COLOR', 'name': 'Color', 'value_id': ...",http://http2.mlstatic.com/D_864629-MCO40475967...,210000
998,MCO542656528,Fantasia Walt Disney Video Laserdisc- Musical ...,"[{'id': 'ITEM_CONDITION', 'name': 'Condición d...",http://http2.mlstatic.com/D_841362-MCO28010690...,55000


## Revisión en detalle de los atributos de los productos

In [25]:
# Suponiendo que 'df' es el DataFrame obtenido en la extracción inicial de la API

# Extraer el contenido del campo 'attributes' del primer producto
first_product_attributes = df_pertinent['attributes'].iloc[0]

print("Explorando los atributos del primer producto:")
print("-" * 50)

# Iterar sobre cada atributo y mostrar la información clave
for attr in first_product_attributes:
    attr_id = attr.get('id')
    attr_name = attr.get('name')
    value_id = attr.get('value_id')
    value_name = attr.get('value_name')
    value_type = attr.get('value_type')
    value_struct = attr.get('value_struct')
    
    print(f"Attribute ID: {attr_id}")
    print(f"Name: {attr_name}")
    print(f"Value ID: {value_id}")
    print(f"Value Name: {value_name}")
    print(f"Value Type: {value_type}")
    print(f"Value Struct: {value_struct}")
    print("-" * 50)


Explorando los atributos del primer producto:
--------------------------------------------------
Attribute ID: BRAND
Name:  Marca del producto electrónico
Value ID: None
Value Name: CERCAS DEL ORIENTE
Value Type: string
Value Struct: None
--------------------------------------------------
Attribute ID: GTIN
Name: Código universal de producto
Value ID: None
Value Name: 000223213232,000055393829
Value Type: string
Value Struct: None
--------------------------------------------------
Attribute ID: ITEM_CONDITION
Name: Condición del ítem
Value ID: 2230284
Value Name: Nuevo
Value Type: list
Value Struct: None
--------------------------------------------------
Attribute ID: PACKAGE_LENGTH
Name: Largo del paquete
Value ID: None
Value Name: 21.4 cm
Value Type: number_unit
Value Struct: {'number': 21.4, 'unit': 'cm'}
--------------------------------------------------
Attribute ID: PACKAGE_WEIGHT
Name: Peso del paquete
Value ID: None
Value Name: 1180 g
Value Type: number_unit
Value Struct: {'num

In [26]:
from collections import Counter

# Inicializar un contador global para los atributos (ID y nombre)
global_attribute_counter = Counter()

# Inicializar un diccionario para almacenar, por producto, la lista de atributos (ID y nombre)
product_attribute_stats = {}

# Recorrer cada producto del DataFrame
for idx, row in df_pertinent.iterrows():
    product_id = row['id']
    
    # Verificar que 'attributes' sea una lista
    if isinstance(row['attributes'], list):
        # Extraer todos los IDs y nombres de atributos del producto
        attributes = [(attr.get('id', 'N/A'), attr.get('name', 'N/A')) for attr in row['attributes']]
        # Almacenar la lista de atributos (ID y nombre) para este producto
        product_attribute_stats[product_id] = attributes
        # Actualizar el contador global con estos atributos
        global_attribute_counter.update(attributes)
    else:
        product_attribute_stats[product_id] = []

# Ordenar la frecuencia global de atributos (ID y nombre) de mayor a menor
sorted_attributes = sorted(global_attribute_counter.items(), key=lambda x: x[1], reverse=True)

# Imprimir la frecuencia global de atributos (ID y nombre) ordenada
print("Frecuencia global de atributos (ID y nombre) en el dataset (ordenada):")
for (attr_id, attr_name), count in sorted_attributes:
    print(f"ID: {attr_id} | Nombre: {attr_name} | Frecuencia: {count}")

print("\nEstadísticas por producto (primeros 5 productos):")
for product_id, attributes in list(product_attribute_stats.items())[:5]:
    print(f"Producto {product_id} - Cantidad de atributos: {len(attributes)}")
    print(f"  Atributos (ID y nombre): {attributes}")
    print("-" * 50)


Frecuencia global de atributos (ID y nombre) en el dataset (ordenada):
ID: ITEM_CONDITION | Nombre: Condición del ítem | Frecuencia: 1000
ID: BRAND | Nombre:  Marca del producto electrónico | Frecuencia: 945
ID: GTIN | Nombre: Código universal de producto | Frecuencia: 401
ID: MODEL | Nombre: Modelo | Frecuencia: 255
ID: PACKAGE_LENGTH | Nombre: Largo del paquete | Frecuencia: 109
ID: PACKAGE_WEIGHT | Nombre: Peso del paquete | Frecuencia: 109
ID: COLOR | Nombre: Color | Frecuencia: 23
ID: UNITS_PER_PACK | Nombre: Unidades por pack | Frecuencia: 6
ID: UNITS_PER_PACKAGE | Nombre: Unidades por envase | Frecuencia: 4
ID: CABLE_LENGTH | Nombre: Largo del cable | Frecuencia: 3
ID: MAIN_COLOR | Nombre: Color principal | Frecuencia: 3
ID: LINE | Nombre: Línea | Frecuencia: 2
ID: GENDER | Nombre: Género | Frecuencia: 2
ID: POWER | Nombre: Potencia | Frecuencia: 2
ID: WEIGHT | Nombre: Peso | Frecuencia: 2
ID: DETAILED_MODEL | Nombre: Modelo detallado | Frecuencia: 1
ID: LIGHT_COLOR | Nombre: Co

## Selección de Atributos Basada en Cobertura
Cobertura de un atributo

Es la proporción de registros (productos) que tienen un valor no nulo para ese atributo. Ejemplo: Si solo el 2% de los productos tiene el atributo “Color”, su cobertura es baja.
Importancia de la cobertura

Algoritmos de clustering necesitan información consistente para identificar patrones.
Atributos con baja cobertura (<10%) suelen:
Aportar poca diferenciación.
Generar problemas debido a valores nulos.
Criterio inicial de selección

Filtrar atributos con baja cobertura simplifica la matriz de datos y reduce la carga de imputaciones.
Atributos con una cobertura superior al umbral mínimo (p. ej., 10-30%) deben analizarse en función de su relevancia para el objetivo analítico.

In [27]:
import pandas as pd

# Lista de atributos de interés
attributes_of_interest = ["ITEM_CONDITION", "BRAND", "GTIN", "MODEL", "PACKAGE_LENGTH", "PACKAGE_WEIGHT"]

# Diccionario para almacenar estadísticas
attribute_stats = {}

# Iterar sobre cada producto para analizar los atributos en la columna 'attributes'
for _, row in df_pertinent.iterrows():
    if isinstance(row['attributes'], list):  # Verificar si 'attributes' es una lista
        for attr in row['attributes']:
            attr_id = attr.get('id')
            attr_value = attr.get('value_name')

            if attr_id in attributes_of_interest:
                # Si el atributo está en la lista de interés, actualizar estadísticas
                if attr_id not in attribute_stats:
                    attribute_stats[attr_id] = {
                        "name": attr.get('name', 'N/A'),
                        "values": [],
                        "non_null_count": 0,
                        "null_count": 0
                    }
                if attr_value:
                    attribute_stats[attr_id]["values"].append(attr_value)
                    attribute_stats[attr_id]["non_null_count"] += 1
                else:
                    attribute_stats[attr_id]["null_count"] += 1

# Calcular estadísticas finales
for attr_id, stats in attribute_stats.items():
    unique_values = set(stats["values"])
    stats["unique_count"] = len(unique_values)
    stats["total_count"] = stats["non_null_count"] + stats["null_count"]
    stats["null_percentage"] = (stats["null_count"] / stats["total_count"]) * 100
    stats["unique_values"] = list(unique_values)  # Opcional: lista de valores únicos

# Crear un DataFrame con las estadísticas
stats_df = pd.DataFrame.from_dict(attribute_stats, orient="index")
stats_df = stats_df[["name", "total_count", "non_null_count", "null_count", "null_percentage", "unique_count"]]

print(stats_df)


                                           name  total_count  non_null_count  \
BRAND            Marca del producto electrónico          945             920   
GTIN               Código universal de producto          401             323   
ITEM_CONDITION               Condición del ítem         1000            1000   
PACKAGE_LENGTH                Largo del paquete          109             109   
PACKAGE_WEIGHT                 Peso del paquete          109             109   
MODEL                                    Modelo          255             250   

                null_count  null_percentage  unique_count  
BRAND                   25         2.645503           450  
GTIN                    78        19.451372            98  
ITEM_CONDITION           0         0.000000             3  
PACKAGE_LENGTH           0         0.000000            77  
PACKAGE_WEIGHT           0         0.000000            66  
MODEL                    5         1.960784           232  


In [28]:
from collections import defaultdict, Counter

# Inicializar un diccionario para almacenar los valores únicos y sus frecuencias por atributo
attributes_of_interest = ["BRAND", "GTIN", "ITEM_CONDITION", "PACKAGE_LENGTH", "PACKAGE_WEIGHT", "MODEL"]
attribute_values = {attr: Counter() for attr in attributes_of_interest}

# Iterar sobre las filas del DataFrame
for _, row in df_pertinent.iterrows():
    if isinstance(row['attributes'], list):  # Verificar si 'attributes' es una lista
        for attr in row['attributes']:
            attr_id = attr.get('id')
            value_name = attr.get('value_name')
            # Si el atributo está en los de interés, contar el valor
            if attr_id in attributes_of_interest and value_name:
                attribute_values[attr_id][value_name] += 1

# Mostrar los resultados
for attr, counter in attribute_values.items():
    print(f"\nValores únicos de {attr} y sus cantidades (ordenados):")
    # Ordenar por frecuencia de mayor a menor
    sorted_values = counter.most_common()
    
    # Mostrar valores con más de una ocurrencia
    for value, count in sorted_values:
        if count > 1:
            print(f"- {value}: {count}")
    
    # Mostrar un resumen de los valores únicos con solo una ocurrencia
    single_occurrences = [value for value, count in sorted_values if count == 1]
    if single_occurrences:
        print(f"- Resto ({len(single_occurrences)} valores con una sola ocurrencia)")
    print("-" * 50)



Valores únicos de BRAND y sus cantidades (ordenados):
- Luxury: 135
- Alpha: 36
- DEL AGRO SOLUCIONES: 29
- Genérica: 28
- Inti: 26
- THE RAY DRIVEN: 22
- Electro TNT: 14
- Excelite: 13
- JR CERCA ELECTRICA: 10
- TNT Electronics: 8
- Compu tools: 8
- Super Fox: 7
- Made in China: 6
- inti: 6
- Copperwell: 6
- Camco: 6
- RL: 5
- Lhaura: 5
- Cnc Maquina Láser Espejo: 5
- Ultra: 5
- Timer: 5
- Performance Tool: 5
- Bold: 4
- JR CERCA ELÉCTRICA: 4
- Wgingenieria: 4
- Cnc Láser Maquina: 4
- Inti Solar: 4
- Turnigy: 4
- Agrofer: 4
- Génerico: 4
- Navcar: 4
- Leviton: 4
- ARP: 4
- Unitec: 3
- Sumsour: 3
- RaveStore: 3
- Skulltrap Electronics: 3
- Lmg: 3
- Pattex: 3
- Abro: 3
- Eym: 3
- One Pixel: 3
- Powest: 3
- Solar Plus: 3
- LMG: 3
- Globy: 3
- WEP: 3
- Lahura Vet: 3
- Energy Suspension: 3
- Aeroquip: 3
- Estilos Myf: 2
- Litio: 2
- Restarsolar: 2
- Universal: 2
- Ya Xun: 2
- Alla France: 2
- Mechanic: 2
- Pelv: 2
- LedPro_Col: 2
- CRC: 2
- Solar 6A: 2
- Hitronic: 2
- Cnc láser: 2
- TURNE

**Razones de aceptación o descarte de los atributos:**

1. **Atributo `BRAND`**  
   - **Aceptado** debido a su amplia cobertura y a que presenta valores repetidos, lo cual permite identificar grupos de artículos que comparten la misma marca.

2. **Atributo `GTIN`**  
   - **Aceptado** en aquellos casos en que aparezca repetido, pues actúa como referencia confiable para unir artículos equivalentes, incluso si los venden distintos vendedores. Aunque no lo reporten todos los productos, sigue siendo muy valioso para agrupar los que sí lo tienen.

3. **Atributo `ITEM_CONDITION`**  
   - **Descartado** por estar demasiado sesgado hacia “Nuevo” y aportar poca diferenciación global en el conjunto.

4. **Atributos `PACKAGE_LENGTH` y `PACKAGE_WEIGHT`**  
   - **Descartados** por la baja cobertura (alrededor de 10%) y por la gran dispersión de valores, que dificulta su uso para la mayoría de los productos.

5. **Atributo `MODEL`**  
   - **Descartado** debido a la cobertura limitada (aproximadamente 25%) y a que la mayoría de los valores son únicos por producto, lo que impide una agrupación significativa en la práctica.

In [29]:
# Atributos seleccionados para incluir (aceptados)
accepted_attributes = ["BRAND", "GTIN"]

# Función para extraer los valores de atributos seleccionados
def extract_attribute_value(attributes, attribute_id):
    if isinstance(attributes, list):
        for attr in attributes:
            if attr.get('id') == attribute_id:
                return attr.get('value_name')  # Extrae el valor legible del atributo
    return None  # Si no se encuentra el atributo, devuelve None

# Crear un nuevo DataFrame basado en df_pertinent con los atributos aceptados
df_propuesto = df_pertinent.copy()

# Agregar columnas para los atributos aceptados
for attribute_id in accepted_attributes:
    df_propuesto[attribute_id.lower()] = df_propuesto['attributes'].apply(lambda x: extract_attribute_value(x, attribute_id))

# Eliminar la columna 'attributes' ya que no se requiere más
df_propuesto.drop(columns=['attributes'], inplace=True)

# Mostrar el DataFrame propuesto al usuario
df_propuesto

,id,title,thumbnail,price,brand,gtin
0,MCO1312345739,Cerca 75km - Ganado - Impulsor,http://http2.mlstatic.com/D_926804-MCO70350738...,165000,CERCAS DEL ORIENTE,"000223213232,000055393829"
1,MCO1401210195,Planta Cerca Electrica Para Ganado 50km M. Jr,http://http2.mlstatic.com/D_802783-MCO74899505...,140000,JR CERCA ELECTRICA,None
2,MCO601595467,Limpiador Electrónico -removedor De Polvo - A...,http://http2.mlstatic.com/D_995727-MCO73203873...,21500,ABRO DUSTER,None
3,MCO611865640,"Planta Cerca Eléctrica Solar 100km, Incluye Pa...",http://http2.mlstatic.com/D_844953-MCO45203598...,690000,UNIVERSAL ELECTRÓNICS,None
4,MCO599963912,Tensor Metálico Para Cerca Eléctrica Para 500 ...,http://http2.mlstatic.com/D_797256-MCO71264690...,6600,Luxury,7707314156920
...,...,...,...,...,...,...
995,MCO542097832,Gripple Plus Medium Pack De 20 | Alambre Joine...,http://http2.mlstatic.com/D_926412-MCO44224848...,274000,Yardware etcetera,"672205002204,0672205002204"
996,MCO542414089,Presurizador Bomba Automatico Soluciona La Baj...,http://http2.mlstatic.com/D_914137-MCO45760245...,379999,Dmgas,None
997,MCO542469534,"8,5 X 11 Sostenedor De La Muestra De Acrílico ...",http://http2.mlstatic.com/D_864629-MCO40475967...,210000,Arraview,"605930132700,0605930132700"
998,MCO542656528,Fantasia Walt Disney Video Laserdisc- Musical ...,http://http2.mlstatic.com/D_841362-MCO28010690...,55000,None,None


In [30]:
df_propuesto.to_csv("../data/df_propuesto.csv", index=False, encoding="utf-8")
